In [2]:
!pip install pytesseract pillow
!pip install python-telegram-bot==13.15 tornado==6.1
!sudo apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
# Importações
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from PIL import Image, ImageEnhance
import pytesseract
import torch


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Caminhos do projeto
model_path = "/content/drive/MyDrive/TrabalhoIA2/Modelos/cardiffnlp_twitter-roberta-base-sentiment"

In [6]:
# Carrega o tokenizador e o modelo treinado
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [7]:
# Função para prever o sentimento do texto
def predict_sentiment(text):
    # Tokeniza o texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Realiza a inferência
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Mapeia os rótulos
    labels = ["Negative", "Neutral", "Positive"]

    # Identifica o rótulo com maior probabilidade
    predicted_label = labels[probs.argmax().item()]
    return predicted_label, probs

In [8]:
# Função para extrair texto de uma imagem
def extract_text_from_image(image_path):
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return None

In [9]:
# Comando /start para iniciar o bot
def start(update, context):
    update.message.reply_text("Hello! Send me a text or an image with English text, and I'll tell you if it's Positive, Neutral, or Negative sentiment.")

In [10]:
# Resposta às mensagens de texto
def handle_text(update, context):
    user_text = update.message.text.strip()

    # Preve o sentimento
    sentiment, probabilities = predict_sentiment(user_text)

    # Formata a resposta
    response = (
        f"Sentiment: {sentiment}\n"
        f"Probabilities:\n"
        f"  - Negative: {probabilities[0][0].item():.2f}\n"
        f"  - Neutral: {probabilities[0][1].item():.2f}\n"
        f"  - Positive: {probabilities[0][2].item():.2f}"
    )
    update.message.reply_text(response)

In [11]:
# Resposta às imagens enviadas
def handle_image(update, context):
    photo = update.message.photo[-1]
    photo_file = context.bot.get_file(photo.file_id)
    image_path = "/tmp/image.jpg"
    photo_file.download(image_path)
    extracted_text = extract_text_from_image(image_path)
    if extracted_text:
        sentiment, probabilities = predict_sentiment(extracted_text)
        response = (
            f"Extracted Text: {extracted_text}\n\n"
            f"Sentiment: {sentiment}\n"
            f"Probabilities:\n"
            f"  - Negative: {probabilities[0][0].item():.2f}\n"
            f"  - Neutral: {probabilities[0][1].item():.2f}\n"
            f"  - Positive: {probabilities[0][2].item():.2f}"
        )
    else:
        response = "I couldn't extract any text from the image. Please try again with a clearer image."
    update.message.reply_text(response)

In [12]:
# Função principal para rodar o bot
def main():
    TELEGRAM_TOKEN = "YOUR-TOKEN-HERE"

    # Configura o Updater e Dispatcher
    updater = Updater(TELEGRAM_TOKEN, use_context=True)
    dp = updater.dispatcher

    # Adiciona manipuladores de comandos e mensagens
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_text))
    dp.add_handler(MessageHandler(Filters.photo, handle_image))

    print("Em execução...")

    # Inicia o bot
    updater.start_polling()
    updater.idle()

In [ ]:
# Executa o bot
main()

Em execução...
